# Inspect Chroma Database


In [1]:
import chromadb
from chromadb.config import Settings
from pathlib import Path
import pandas as pd
import json
from src.app.config.settings import settings


## Conectar ao ChromaDB

In [2]:
client = chromadb.PersistentClient(
    path=str(settings.chroma_dir),
    settings=Settings(anonymized_telemetry=False)
)
client

## Listar coleções disponíveis

In [3]:
collections = client.list_collections()
collections

[Collection(name=documents)]

##  Abrir coleção principal

In [13]:
collection = client.get_collection(settings.chroma_collection)
collection

Collection(name=documents)

##  Contar quantos documentos existem no banco

In [14]:
count = collection.count()
count

539

##  Carregar todos os documentos (cuidado em base muito grande)

In [15]:
all_docs = collection.get(include=["documents", "metadatas", "embeddings"])
len(all_docs["documents"])

539

##  Mostrar preview dos primeiros 5 documentos

In [16]:
df_preview = pd.DataFrame({
    "text": all_docs["documents"][:5],
    "metadata": all_docs["metadatas"][:5]
})
df_preview

,text,metadata
0,(Mark One) \n☒ ANNUAL REPORT PURSUANT TO SECT...,{'doc_id': 'fd8502ebf9d3a72c3a5e95308a59589d14...
1,| Title of each class |...,{'doc_id': 'fd8502ebf9d3a72c3a5e95308a59589d14...
2,| 3.600% Notes due 2042 |...,"{'section_path': '', 'order': 2, 'doc_id': 'fd..."
3,## UNITED STATES SECURITIES AND EXCHANGE COMMI...,{'section_path': 'UNITED STATES SECURITIES AND...
4,## FORM 10-K \nIndicate by check mark whether...,"{'doc_date': '', 'section_path': 'FORM 10-K', ..."


## Ver todas as chaves de metadados existentes

In [8]:
metadata_keys = set()
for meta in all_docs["metadatas"]:
    metadata_keys |= set(meta.keys())
metadata_keys

{'chunk_kind',
 'company',
 'doc_date',
 'doc_id',
 'doc_type',
 'order',
 'section_path'}

## Filtrar documentos por doc_id

In [9]:
def find_by_doc_id(doc_id):
    docs = collection.get(
        where={"doc_id": {"$eq": doc_id}},
        include=["documents", "metadatas"]
    )
    return pd.DataFrame({
        "text": docs["documents"],
        "metadata": docs["metadatas"]
    })

# Exemplo:
# find_by_doc_id("nvidia-q3-2024")

##  Amostrar chunks aleatórios

In [10]:
import random

def sample_chunks(n=3):
    idxs = random.sample(range(len(all_docs["documents"])), min(n, len(all_docs["documents"])))
    data = []
    for i in idxs:
        data.append({
            "text": all_docs["documents"][i],
            "metadata": all_docs["metadatas"][i]
        })
    return pd.DataFrame(data)

sample_chunks(3)

,text,metadata
0,## Comitê de Política Monetária -Copom \n10 e...,"{'chunk_kind': 'text', 'section_path': 'Comitê..."
1,| iTunes K.K. ...,"{'doc_type': '10-Q Filing', 'order': 388, 'com..."
2,## Macroeconomic and Industry Risks,"{'company': 'Apple', 'doc_date': '', 'doc_id':..."


## 🔟 Ver dados agrupados por seção (section_path)

In [11]:
def find_by_section(section: str):
    docs = collection.get(
        where={"section_path": {"$eq": section}},
        include=["documents", "metadatas"]
    )
    return pd.DataFrame({
        "text": docs["documents"],
        "metadata": docs["metadatas"]
    })

# Exemplo:
# find_by_section("Financial Statements > Revenue")

## 1️⃣1️⃣ Ver formato dos embeddings

In [12]:
emb = all_docs["embeddings"][0]
len(emb), emb[:10]

(1536,
 array([ 0.02699038,  0.01287768,  0.03478646,  0.03010367,  0.01716166,
        -0.00547398,  0.00225295, -0.00515236, -0.01037547,  0.01525767]))

## 1️⃣2️⃣ Análise Completa dos Metadados

In [17]:
# Análise detalhada de cada chave de metadado
metadata_analysis = {}

for key in sorted(metadata_keys):
    values = [meta.get(key) for meta in all_docs["metadatas"] if key in meta]
    unique_values = set(values)
    
    metadata_analysis[key] = {
        "total_ocorrências": len(values),
        "valores_únicos": len(unique_values),
        "exemplos": list(unique_values)[:5],  # Até 5 exemplos
        "tipo_dados": type(values[0]).__name__ if values else "None"
    }

# Mostrar análise formatada
for key, info in metadata_analysis.items():
    print(f"\n📌 '{key}':")
    print(f"   Tipo: {info['tipo_dados']}")
    print(f"   Ocorrências: {info['total_ocorrências']}")
    print(f"   Valores únicos: {info['valores_únicos']}")
    print(f"   Exemplos: {info['exemplos']}")



📌 'chunk_kind':
   Tipo: str
   Ocorrências: 539
   Valores únicos: 2
   Exemplos: ['text', 'table']

📌 'company':
   Tipo: str
   Ocorrências: 539
   Valores únicos: 6
   Exemplos: ['', 'Microsoft', 'Apple', 'Banco Central do Brasil', 'NVIDIA']

📌 'doc_date':
   Tipo: str
   Ocorrências: 539
   Valores únicos: 3
   Exemplos: ['', 'November 2023', '2023-11-21']

📌 'doc_id':
   Tipo: str
   Ocorrências: 539
   Valores únicos: 6
   Exemplos: ['c013e7147789ebe5be4b8b2d52d4083a1134056d', '76069ec81d26753e4be5042f2ef3ee6b7d1d6e88', '588ad141573701c6a100a73ecfc8a0d2a4f2436a', 'fd8502ebf9d3a72c3a5e95308a59589d14c91c7d', '25fa12860dc1573585f15942e9b1b6d0620581d4']

📌 'doc_type':
   Tipo: str
   Ocorrências: 539
   Valores únicos: 3
   Exemplos: ['10-Q Filing', 'Research Report', 'Meeting Minutes']

📌 'order':
   Tipo: int
   Ocorrências: 539
   Valores únicos: 399
   Exemplos: [0, 1, 2, 3, 4]

📌 'section_path':
   Tipo: str
   Ocorrências: 539
   Valores únicos: 359
   Exemplos: ['', '2.  The

## 1️⃣3️⃣ Tabela Resumida dos Metadados

In [ ]:
# Criar dataframe com resumo dos metadados
df_metadata_summary = pd.DataFrame([
    {
        "Chave": key,
        "Tipo": info["tipo_dados"],
        "Ocorrências": info["total_ocorrências"],
        "Valores Únicos": info["valores_únicos"],
        "Cobertura %": f"{(info['total_ocorrências']/len(all_docs['metadatas'])*100):.1f}%"
    }
    for key, info in metadata_analysis.items()
])

df_metadata_summary.sort_values("Ocorrências", ascending=False)

## 1️⃣4️⃣ Exemplos Reais de Metadados Completos

In [ ]:
# Mostrar metadados completos de alguns documentos
print("=" * 80)
print("EXEMPLOS DE METADADOS COMPLETOS:")
print("=" * 80)

for i in range(min(3, len(all_docs["metadatas"]))):
    print(f"\n📄 Documento #{i+1}:")
    print(f"Texto (primeiros 150 chars): {all_docs['documents'][i][:150]}...")
    print(f"\nMetadados:")
    for key, value in all_docs["metadatas"][i].items():
        print(f"  • {key}: {value}")
    print("-" * 80)

## 1️⃣5️⃣ Exportar Metadados para JSON

In [ ]:
# Exportar metadados em JSON para análise completa
output_file = Path("data/staged/metadados_completos.json")
output_file.parent.mkdir(parents=True, exist_ok=True)

# Estruturas úteis para exportar
export_data = {
    "total_documentos": len(all_docs["documents"]),
    "total_chunks": len(all_docs["documents"]),
    "chaves_metadados": sorted(list(metadata_keys)),
    "analise_chaves": metadata_analysis,
    "amostra_metadados": all_docs["metadatas"][:10]  # Primeiros 10 documentos
}

with open(output_file, "w") as f:
    json.dump(export_data, f, indent=2, ensure_ascii=False)

print(f"✅ Metadados exportados para: {output_file}")
print(f"Arquivo criado com {len(export_data)} seções principais")
print(f"\nConteúdo resumido:")
for section, value in export_data.items():
    if isinstance(value, (list, dict)):
        print(f"  • {section}: {len(value)} itens")
    else:
        print(f"  • {section}: {value}")